In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import torch
import lpips
from torchvision import transforms
from dataloaders import Kodak, CLIC, DIV2K
from PIL import Image
import yaml
from pytorch_msssim import ms_ssim
import glob
import matplotlib.pyplot as plt
from prompt_inversion.optim_utils import clip_cosine, read_json
import prompt_inversion.open_clip as open_clip
from argparse import Namespace
import tqdm

device = 'cuda:1'

In [3]:
args = Namespace()
args.__dict__.update(read_json("prompt_inversion/sample_config.json"))

clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(args.clip_model, pretrained=args.clip_pretrain, device=device)

def distortion_CLIP(x, xhat):
    return 1 - clip_cosine([x], [xhat], clip_model, clip_preprocess, device)

In [4]:
lpips_loss = lpips.LPIPS(net='alex') 

def ntc_preprocess(image):
    # transform = transforms.Compose(
    #         [transforms.Grayscale(), transforms.ToTensor()]
    #     )
    transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )
    image = transform(image)
    return image

def distortion_LPIPS(orig, recon):
    recon = ntc_preprocess(recon)
    orig = ntc_preprocess(orig)

    return lpips_loss(orig.unsqueeze(0), recon.unsqueeze(0)).item()
    # loss += lpips_loss(orig.unsqueeze(0), recon.unsqueeze(0)).item()
    

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/eric/miniconda3/envs/control2/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/eric/miniconda3/envs/control2/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/eric/miniconda3/envs/control2/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [5]:
def ntc_preprocess(image):
    # transform = transforms.Compose(
    #         [transforms.Grayscale(), transforms.ToTensor()]
    #     )
    transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )
    image = transform(image)
    return image

In [7]:
# dm = Kodak(root='/home/Shared/image_datasets/Kodak/', batch_size=1)
# dm = CLIC(root='/home/Shared/image_datasets/CLIC_resized/2021', batch_size=1)
dm = DIV2K(root='/home/Shared/image_datasets/DIV2K_resized', batch_size=1)
dset = dm.test_dset

In [15]:
# name = 'DIV2K/q1_ms_ssim'
# exp = 'mbt2018_lowrate'
name = 'SD_pi_lpips'

loss = 0
bpp = 0
for i, im in tqdm.tqdm(enumerate(dset), total=len(dset)):
    recon = Image.open(f'recon_examples/{exp}/{name}/{i}_recon.png')
    orig = transforms.functional.to_pil_image(im[0])
    # loss += distortion_CLIP(orig, recon)
    loss += distortion_LPIPS(orig, recon)
    # loss += ms_ssim(orig.unsqueeze(0), recon.unsqueeze(0), 1).item()

    fpath = glob.glob(f'recon_examples/{exp}/{name}/{i}*.yaml')[0]
    with open(fpath, 'r') as stream:
        f = yaml.safe_load(stream)
    bpp += f['bpp_total']

print(f'Avg loss={loss/len(dset)}, bpp={bpp/len(dset)}')

100%|██████████| 100/100 [00:06<00:00, 15.61it/s]

Avg loss=0.5339853346347809, bpp=0.0374881085634946


In [16]:
print(f'Avg loss={loss/len(dset)}, bpp={bpp/len(dset)}')

Avg loss=0.5339853346347809, bpp=0.0374881085634946


: 

In [ ]:

# CLIC
# q1_ms-ssim: bpp=0.0320, LPIPS=0.44573, FID=306.6994, KID=0.07609493, CLIP=0.323728
# q2_ms-ssim: bpp=0.0498, LPIPS=0.3798, FID=284.1792, KID=0.05331241, CLIP=0.25363
# SD_pi: bpp=0.00283, LPIPS=0.74414, FID=246.7897, KID=0.01864557, CLIP=0.32015
# SD_pi+hed_lpips_rcc: bpp=0.01507, LPIPS=0.62328, FID=267.0152, KID=0.03444665, CLIP=0.2658

# DIV2K
# q1_ms-ssim: bpp=0.0375, LPIPS=0.5340, FID=239.9637, KID=0.0505623, CLIP=0.3425
# q2_ms-ssim: bpp=0.06014, LPIPS=0.456, FID=213.7037, KID=0.02861265, CLIP=0.2699
# SD_pi: bpp=0., LPIPS=0., FID=, KID=0., CLIP=0.
# SD_pi+hed_lpips_rcc: bpp=0., LPIPS=0., FID=, KID=0., CLIP=0.
